# New try here using simpy and optimize, starting with the cart example and coding everything new

## Reference of sympy functionality

In [ ]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
# x, y, z = sympy.symbols("x y z")
# parameters = {"T":sympy.cos(3.4,evaluate=False)}
# def functest(x,y,c,params):
#     return x + x*y + params["T"]


# expr = sympy.Derivative(functest(x,y,z,parameters),x,evaluate=True) + 1

# func_eval = lambda func,b: func.evalf(subs={x:b,y:b})

# def func_test_2(q,params):
#     q_x, q_y = q
#     return params["T"]*q_x* q_y + 2* q_y + params["T"]


In [ ]:
# testvec = np.array([x,y])
# expr_2 = sympy.Derivative(func_test_2(testvec,parameters),y)
# print(expr_2)
# expr_2.doit()

In [ ]:
# from sympy import symbols, lambdify
# from sympy.parsing.sympy_parser import parse_expr
# import numpy as np

# xs = symbols('x1 x2')
# ks = symbols('k1 k2')
# strs = ['-k1 * x1**2 + k2 * x2', 'k1 * x1**2 - k2 * x2']
# syms = [parse_expr(item) for item in strs]

# # Convert each expression in syms to a function with signature f(x1, x2, k1, k2):
# funcs = [lambdify(xs + ks, f) for f in syms]


# # This is not exactly the same as the `my_odes` in the question.
# # `t` is included so this can be used with `scipy.integrate.odeint`.
# # The value returned by `sym.subs` is wrapped in a call to `float`
# # to ensure that the function returns python floats and not sympy Floats.
# def my_odes(x, t, k):
#     all_dict = dict(zip(xs, x))
#     all_dict.update(dict(zip(ks, k)))
#     return np.array([float(sym.subs(all_dict)) for sym in syms])

# def lambdified_odes(x, t, k):
#     x1, x2 = x
#     k1, k2 = k
#     xdot = [f(x1, x2, k1, k2) for f in funcs]
#     return xdot


# if __name__ == "__main__":
#     from scipy.integrate import odeint

#     k1 = 0.5
#     k2 = 1.0
#     init = [1.0, 0.0]
#     t = np.linspace(0, 1, 6)
#     sola = odeint(lambdified_odes, init, t, args=((k1, k2),))
#     solb = odeint(my_odes, init, t, args=((k1, k2),))
#     print(np.allclose(sola, solb))

# Helper functions

In [ ]:
def weighted_sum(x1,x2,weight):
    return weight * x1 + (1-weight)*x2

def finite_diff(x1,x2,step_size):
    return (x2-x1)/step_size


# Solving simple driven cart OCP
2D Problem

In [ ]:

# u,v = sympy.var('u v')
# x = (2 + sympy.sin(u) *sympy.sin(v)) *sympy.sin(3*v/2) 
# y = sympy.cos(u) *sympy.sin(v) + 2 *v/sympy.pi - 2 
# z = (2 + sympy.sin(u) *sympy.sin(v)) *sympy.cos(3*v/2)
   # everything so far was as in your code 
# vec =sympy.Matrix([u,v])
# print(vec)
# print(vec.transpose().multiply(vec))    # "pretty" print
# print(sympy.Matrix([u]))
# def functest(vec):
#    a,b = vec
#    return a+b

# sympy.Derivative(functest(vec),"u",evaluate=True)
# len(OCP_parameters)

In [ ]:
numpier = sympy.symbols("nper")
to_numpy_transformer = lambda x : sympy.lambdify(numpier,x)
eval_sympy_to_np = lambda x:  to_numpy_transformer(x)(0).flatten()
# eval_sympy_to_np(OCP_parameters["q0"])

In [ ]:
import sympy
import numpy as np
import scipy.optimize as opt
# x = sympy.symbols("x")
# v_x = sympy.symbols("v_x")
# q = sympy.Matrix([x])

###################################
#### Emergency solution via these params, for that it works
# OCP_parameters = {
#     "m1":10.,  # kg
#     "m2": 9.,
#     "length": 0.5,  # m (rod length)
#     "g_constant": 9.81, #m/s**2
#     "I": 3.2, #  kg * m^2
#     "Aq": 50., #  kg/s^2
#     "Adq": 90., # kg/s
#     "q0": sympy.Matrix([0.,0]),        #(m,1)
#     "dq0": sympy.Matrix([0.,0.]),       #(m/s,1/s)
#     "qT": sympy.Matrix([1.,0.]),       #(m,1)
#     "dqT": sympy.Matrix([0.,0.]),       #(m/s,1/s)
#     "alpha": 0.5,
#     "gamma": 0.5,  
#     "T": 16.,   #s
#     "N": 80,
#     "dim_q": 2,  # (x,y)
#     "dim_u": 1,  
# }
#######################################

OCP_parameters = {
    "m1":10.,  # kg
    "m2": 9.,
    "length": 0.5,  # m (rod length)
    "g_constant": 9.81, #m/s**2
    "I": 3.2, #  kg * m^2
    "Aq": 123.70, #  kg/s^2
    "Adq": 127.70, # kg/s
    "q0": sympy.Matrix([0.,0]),        #(m,1)
    "dq0": sympy.Matrix([0.,0.]),       #(m/s,1/s)
    "qT": sympy.Matrix([18.,0.]),       #(m,1)
    "dqT": sympy.Matrix([0.,0.]),       #(m/s,1/s)
    "alpha": 0.5,
    "gamma": 0.5,  
    "u_prefactor":0.00052,
    "T": 3.,   #s
    "N":100,     #240, 160, 80, 40, 20
    "dim_q": 2,  # (x,y)
    "dim_u": 1,  
}

OCP_parameters["h"] = OCP_parameters["T"]/ OCP_parameters["N"]
OCP_parameters["times"] = sympy.Matrix([i*OCP_parameters["h"] for i in range(OCP_parameters["N"]+1)])


OCP_parameters_np_version = {}
OCP_parameters_np_version["m1"] = OCP_parameters["m1"]
OCP_parameters_np_version["m2"] = OCP_parameters["m2"]
OCP_parameters_np_version["length"] = OCP_parameters["length"]
OCP_parameters_np_version["I"] = OCP_parameters["I"]
OCP_parameters_np_version["g_constant"] = OCP_parameters["g_constant"]

OCP_parameters_np_version["Aq"] = OCP_parameters["Aq"]
OCP_parameters_np_version["Adq"] = OCP_parameters["Adq"]
OCP_parameters_np_version["alpha"] = OCP_parameters["alpha"]
OCP_parameters_np_version["gamma"] = OCP_parameters["gamma"]
OCP_parameters_np_version["T"] = OCP_parameters["T"]
OCP_parameters_np_version["N"] = OCP_parameters["N"]
OCP_parameters_np_version["dim_q"] = OCP_parameters["dim_q"]
OCP_parameters_np_version["dim_u"] = OCP_parameters["dim_u"]
OCP_parameters_np_version["q0"] = eval_sympy_to_np(OCP_parameters["q0"])
OCP_parameters_np_version["dq0"] = eval_sympy_to_np(OCP_parameters["dq0"])
OCP_parameters_np_version["dqT"] = eval_sympy_to_np(OCP_parameters["dqT"])
OCP_parameters_np_version["qT"] = eval_sympy_to_np(OCP_parameters["qT"])
OCP_parameters_np_version["h"] = OCP_parameters_np_version["T"]/ OCP_parameters_np_version["N"]
OCP_parameters_np_version["times"] = eval_sympy_to_np(OCP_parameters["times"])
OCP_parameters_np_version["u_prefactor"] = OCP_parameters["u_prefactor"]

# OCP_parameters_np_version["times"] = np.array([i*OCP_parameters_np_version["h"] for i in range(OCP_parameters_np_version["N"]+1)])





# v = sympy.Matrix([v_x])
q = sympy.symbols("x,phi")     #position
v_q = sympy.symbols("v_x,v_phi") #velocity
a_q = sympy.symbols("a_x,a_phi")  #acceleration
u = sympy.symbols("u,")



v_q_vec = sympy.Matrix([v_q]).transpose()
q_vec = sympy.Matrix([q]).transpose()
a_q_vec = sympy.Matrix([a_q]).transpose()
u_vec = sympy.Matrix([u]).transpose()


Ua_k = sympy.Matrix(sympy.symbols("U_a_k,"))
Ua_k_1 = sympy.Matrix(sympy.symbols("U_a_km1,"))
Ub_k = sympy.Matrix(sympy.symbols("U_b_k,"))
Ub_k_1 = sympy.Matrix(sympy.symbols("U_b_km1,"))

q_k = sympy.Matrix(sympy.symbols("x_k,phi_k")  )   #position
q_k_1 = sympy.Matrix(sympy.symbols("x_km1,phi_km1") )    #position
q_k1 = sympy.Matrix(sympy.symbols("x_k1,phi_k1") )    #position
v_q_k = sympy.Matrix(sympy.symbols("v_x_k,v_phi_k")) #velocity
v_q_k1 = sympy.Matrix(sympy.symbols("v_x_k1,v_phi_k1")) #velocity
v_q_k_1 = sympy.Matrix(sympy.symbols("v_x_km1,v_phi_km1")) #velocity

## Defining the mechanical system

In [ ]:
def mech_Lagrangian(q,v,params):
    m1,m2 = params["m1"],params["m2"]
    ell = params["length"]
    inertia = params["I"]
    grav = params["g_constant"]
    q_x,q_phi = q
    v_x,v_phi = v
    # return q_x**2 + q_y**2 + q_x**2*q_y**2
    Lagrangian = (m1+m2) * (v_x)**2 / 2  
    Lagrangian += ell*m2 *sympy.cos(q_phi)*v_x*v_phi/2
    Lagrangian += ell**2 *m2 * v_phi**2/8
    Lagrangian += inertia * v_phi**2/2 
    Lagrangian += m2*grav *ell *sympy.cos(q_phi)/2
    return Lagrangian
#use func always for the evaluated sympy symbol 
# (probably classes would be better/more natural here)
mech_Lagrangian_func = mech_Lagrangian(q,v_q,OCP_parameters)
mech_Lagrangian_lambda_func = sympy.lambdify(q+v_q,mech_Lagrangian_func)


def f_L(q,v,u,params):
    control_force =  sympy.Matrix([u[0],0.])  #"hardcoded form of forcing, but ok for 2D here considered"
    return control_force # since 1-dimensional
f_L_func = f_L(q,v_q,u,OCP_parameters)
f_L_lambda_func = sympy.lambdify(q+v_q+u,mech_Lagrangian_func)



Dq_L_mech = sympy.derive_by_array(mech_Lagrangian_func, q)
Dv_L_mech = sympy.derive_by_array(mech_Lagrangian_func, v_q)
Dvv_L_mech_chain  = sympy.derive_by_array(Dv_L_mech,v_q).tomatrix()@ a_q_vec
Dvq_L_mech_chain  = sympy.derive_by_array(Dq_L_mech,q).tomatrix()@ v_q_vec

EL_mech  =  Dvv_L_mech_chain + Dvq_L_mech_chain - sympy.Matrix(Dq_L_mech) - sympy.Matrix(f_L_func)
EL_mech_lambda = sympy.lambdify(q+v_q+a_q+u, EL_mech)


# def mech_Lagrangian(q,v,params):
#     m1 = params["m1"]
#     q_x,q_y = q
#     v_x,v_y = v
#     # return q_x**2 + q_y**2 + q_x**2*q_y**2
#     return m1 * (v_x**2+v_y**2) / 2 
def mech_p(q,v,params):
    return sympy.Matrix(sympy.derive_by_array(mech_Lagrangian(q,v,params),v))

# q = sympy.Matrix(sympy.symbols("x,y"))
# v_q = sympy.Matrix(sympy.symbols("v_x,v_y"))
p_mech = sympy.Matrix(sympy.symbols("p_x,p_phi"))
p_mech_as_v = mech_p(q,v_q,OCP_parameters)
v_as_p_mech = sympy.solve([p_mech[i]-p_mech_as_v[i] for i in range(len(p_mech))], v_q)
inverse_Legendre_v = sympy.Matrix([v_as_p_mech[arg] for arg in v_q])


def discrete_Lagrangian(qk,qk1,params):
    gamma = params["gamma"]
    alpha= params["alpha"]
    weighted_q_g = weighted_sum(qk,qk1,gamma)
    weighted_q_1g = weighted_sum(qk,qk1,1-gamma)
    step_size= params["h"]
    finite_diff_q = finite_diff(qk,qk1,step_size)
    result = step_size* alpha*mech_Lagrangian(weighted_q_g,finite_diff_q,params)
    result += step_size* (1-alpha)*mech_Lagrangian(weighted_q_1g,finite_diff_q,params)
    return result

def discrete_f_L_p(qk,qk1,uka,ukb,params):
    gamma = params["gamma"]
    alpha= params["alpha"]
    weighted_q_g = weighted_sum(qk,qk1,gamma)
    weighted_q_1g = weighted_sum(qk,qk1,1-gamma)
    step_size= params["h"]
    finite_diff_q = finite_diff(qk,qk1,step_size)
    result = step_size * alpha * (1-gamma) * f_L(weighted_q_g,finite_diff_q,uka,params) 
    result += step_size * (1-alpha) *gamma * f_L(weighted_q_1g,finite_diff_q,ukb,params) 
    return result

def discrete_f_L_m(qk,qk1,uka,ukb,params):
    gamma = params["gamma"]
    alpha= params["alpha"]
    weighted_q_g = weighted_sum(qk,qk1,gamma)
    weighted_q_1g = weighted_sum(qk,qk1,1-gamma)
    step_size= params["h"]
    finite_diff_q = finite_diff(qk,qk1,step_size)
    result = step_size * alpha * gamma * f_L(weighted_q_g,finite_diff_q,uka,params) 
    result += step_size * (1-alpha) *(1-gamma) * f_L(weighted_q_1g,finite_diff_q,ukb,params) 
    return result



p_k_m = -  sympy.derive_by_array(discrete_Lagrangian(q_k,q_k1,OCP_parameters),q_k).tomatrix()
p_k_m -= discrete_f_L_m(q_k,q_k1,Ua_k,Ub_k,OCP_parameters)
p_k_p = sympy.derive_by_array(discrete_Lagrangian(q_k_1,q_k,OCP_parameters),q_k).tomatrix()
p_k_p += discrete_f_L_p(q_k_1,q_k,Ua_k_1,Ub_k_1,OCP_parameters)
forced_DEL = -p_k_m + p_k_p
all_elements = list(q_k_1) + list(q_k) + list(q_k1) +list(Ua_k_1) + list(Ua_k) 
all_elements += list(Ub_k_1) + list(Ub_k)
forced_DEL_lambdified = sympy.lambdify(all_elements,forced_DEL)

## need now a numeric way to calculate the initial q_1 from v_0

In [ ]:
simplified_expr=sympy.simplify(p_k_m - p_mech_as_v)
semi_evaluated_expr = lambda dict_to_eval: simplified_expr.evalf(subs=dict_to_eval)


# calc_next_q_val_from_v = sympy.Matrix([sympy.nsolve([p_k_m[i] - p_mech_as_v[i] for i in range(len(p_k_m))],q_k1)[arg] for arg in q_k1])
all_elements = list(v_q) +list(q_k)  + list(Ua_k) + list(Ub_k) 
# calc_next_q_val_from_v_lambdified = sympy.lambdify(all_elements,calc_next_q_val_from_v)

In [ ]:
u_a_val_test = sympy.Matrix([1.])
u_b_val_test = sympy.Matrix([2.])


def numerically_get_q1(eval_dict):
    to_numerically_solve = semi_evaluated_expr(eval_dict)
    return eval_sympy_to_np(sympy.nsolve(to_numerically_solve,q_k1, OCP_parameters_np_version["q0"]))


In [ ]:
v_m_as_discrete_q = sympy.Matrix([v_as_p_mech[tmp]  for tmp in v_q]).evalf(subs={tmp1:tmp2 for (tmp1,tmp2) in zip(p_mech,p_k_m)}|{tmp1:tmp2 for tmp1,tmp2 in zip(q,q_k)})
v_p_as_discrete_q = sympy.Matrix([v_as_p_mech[tmp]  for tmp in v_q]).evalf(subs={tmp1:tmp2 for (tmp1,tmp2) in zip(p_mech,p_k_p)}|{tmp1:tmp2 for tmp1,tmp2 in zip(q,q_k)})

v_m_calc_lambdify = sympy.lambdify(list(q_k)+list(q_k1)+list(Ua_k)+list(Ub_k) ,v_m_as_discrete_q,modules="numpy")
v_p_calc_lambdify = sympy.lambdify(list(q_k_1)+list(q_k)+list(Ua_k_1)+list(Ub_k_1) ,v_p_as_discrete_q,modules="numpy")

def calc_velocities_from_discrete_sol(q_d, Ua_d,Ub_d,params):
    '''assumed already working with numpy arrays'''
    v_d = []
    N_val = params["N"]
    for i in range(N_val):
        v_d.append(v_m_calc_lambdify(*q_d[i],*q_d[i+1], *Ua_d[i],*Ub_d[i]))
    v_d.append(v_p_calc_lambdify(*q_d[N_val-1],*q_d[N_val], *Ua_d[N_val-1],*Ub_d[N_val-1]))
    return np.array(v_d).reshape([-1,2])

# v_d_sol = calc_velocities_from_discrete_sol(q_d_vals,U_a_d_values,U_b_d_values,OCP_parameters_np_version)
# v_m_calc_lambdify


# Now solving the IVP of the cart

In [ ]:
# U_a_d_values_tmp = [0.1 for i in range(int(OCP_parameters["N"]/40))]
# U_a_d_values = []
# for i in range(10):
#     for x in U_a_d_values_tmp:
#         U_a_d_values.append( [x])
#     for x in U_a_d_values_tmp:
#         U_a_d_values.append([-1* x])
#         U_a_d_values.append([-1* x])
#     for x in U_a_d_values_tmp:
#         U_a_d_values.append( [x])
# U_a_d_values= np.array(U_a_d_values)    

In [ ]:
import pickle
import scipy
save_data_file = 'data/new_u_dep_data_a=0.5g=0.5/new_u_dep_data_a=0.5g=0.5N=100.pkl'
with open(save_data_file, 'rb') as files:
    initial_guess_data = pickle.load(files)

# q_d_start_guess = np.array(initial_guess_data['q_d'])
# lam_d_start_guess =np.array(initial_guess_data['lam_d'])
# u_d_start_guess = initial_guess_data['U_d']

# cs_q =  scipy.interpolate.CubicSpline(np.linspace(0,parameters["t_N"],len(q_d_start_guess)), q_d_start_guess.reshape([len(q_d_start_guess),2]))
# cs_lam = scipy.interpolate.CubicSpline(np.linspace(0,parameters["t_N"],len(lam_d_start_guess)), lam_d_start_guess.reshape([len(lam_d_start_guess),2]))  
# cs_u= scipy.interpolate.CubicSpline(np.linspace(0,parameters["t_N"],len(u_d_start_guess)),u_d_start_guess)

# U_d_1_use = cs_u(np.array(parameters["times"]) + parameters["beta"]*parameters["h"]).reshape(len(parameters["times"]),1,1)
# U_d_2_use = cs_u(np.array(parameters["times"]) + (1-parameters["beta"])*parameters["h"]).reshape(len(parameters["times"]),1,1)
# initial_guess_data.keys()
standard_Ua_guess = initial_guess_data['standard_Ua'].flatten()
ua_func = scipy.interpolate.CubicSpline(initial_guess_data['parameters']["times"][:-1], initial_guess_data["standard_Ua"].flatten()) 
ub_func = scipy.interpolate.CubicSpline(initial_guess_data['parameters']["times"][:-1], initial_guess_data["standard_Ub"].flatten()) 

U_a_d_values = np.array(ua_func(OCP_parameters_np_version["times"][:-1] + OCP_parameters_np_version['h']*OCP_parameters_np_version["gamma"])).reshape([-1,1])
U_b_d_values = np.array(ua_func(OCP_parameters_np_version["times"][:-1] + OCP_parameters_np_version['h']*(1.-OCP_parameters_np_version["gamma"]))).reshape([-1,1])


# plt.plot(OCP_parameters_np_version["times"],ub_func(OCP_parameters_np_version["times"]))


In [ ]:
def interval_force(time, intervalLength,t0):
    if time < intervalLength +t0:
        return 1.0
    elif int((time -t0)/intervalLength) %2 == 0:
        return 1.0*int((time -t0)/intervalLength)
    else: 
        return -1.0*int((time -t0)/intervalLength)

# U_a_d_values = np.array([[-np.sin(0.001*i/OCP_parameters["h"])] for i in range(OCP_parameters_np_version["N"]+1)])
# U_a_d_values = 3.5* np.array([[interval_force(OCP_parameters_np_version["times"][i],1.4,0)-0.1] for i in range(OCP_parameters_np_version["N"])])
# U_a_d_values = 0*np.array([[np.cos(OCP_parameters_np_version["times"][i]*2.6)] for i in range(OCP_parameters_np_version["N"])])
# U_a_d_values = np.array([[-i] for i in np.linspace(-230.,230.,OCP_parameters_np_version["N"])])
# U_b_d_values = U_a_d_values

def calculate_full_mech_trajectory(U_a_d_vals,U_b_d_vals):
    q_d_sol = [OCP_parameters_np_version["q0"]]
    # q_d_sol.append(calc_next_q_val_from_v_lambdified(* OCP_parameters_np_version["dq0"],* q_d_sol[0],*U_a_d_vals[0],*U_b_d_vals[0] ).flatten())
    evaluation_dict = {tmp1:tmp2 for tmp1,tmp2 in zip(v_q,OCP_parameters["dq0"])}  | {tmp1:tmp2 for tmp1,tmp2 in zip(q_k,OCP_parameters["q0"])}
    evaluation_dict = evaluation_dict | {tmp1:tmp2 for tmp1,tmp2 in zip(Ua_k,sympy.Matrix(U_a_d_vals[0]))} | {tmp1:tmp2 for tmp1,tmp2 in zip(Ub_k,sympy.Matrix(U_b_d_vals[0]))}
    q_d_sol.append(numerically_get_q1(evaluation_dict))
    for i in range(1,OCP_parameters["N"]):
        eval_func = lambda x: forced_DEL_lambdified(*q_d_sol[i-1],*q_d_sol[i],*x ,*U_a_d_vals[i-1],*U_a_d_vals[i],*U_b_d_vals[i-1],*U_b_d_vals[i] ).flatten()
        test_obj = eval_func(q_d_sol[i]) # not sure if the brackets are correct here for higher dimensions
        result = opt.root(eval_func, q_d_sol[i],method='lm',tol=1e-9)
        if not result.success:
            print('failed eq finding')
        q_d_sol.append(result.x)
    return q_d_sol

q_d_vals = calculate_full_mech_trajectory(U_a_d_values,U_b_d_values)
v_d_sol = calc_velocities_from_discrete_sol(q_d_vals,U_a_d_values,U_b_d_values,OCP_parameters_np_version)
    # print(result)

In [ ]:
plt.plot(OCP_parameters_np_version["times"][:-1],np.array(U_a_d_values).flatten(),label="Ua")
plt.plot(OCP_parameters_np_version["times"][:-1],np.array(U_b_d_values).flatten(),'--',label="Ub")
plt.legend()

In [ ]:
plt.plot(OCP_parameters_np_version["times"],np.array(q_d_vals).transpose()[0].flatten(),label="x")
plt.plot(OCP_parameters_np_version["times"],np.array(q_d_vals).transpose()[1].flatten(),label="phi")
plt.legend()

In [ ]:
plt.plot(OCP_parameters_np_version["times"],v_d_sol.transpose()[0].flatten(),label="v_x")
plt.plot(OCP_parameters_np_version["times"],v_d_sol.transpose()[1].flatten(),label="v_phi")


In [ ]:
#conserved quantity:
def I_eval (q,v,params):
    m1, m2 = params["m1"], params["m2"]
    ell = params["length"]
    x, phi = q
    v_x, v_phi = v
    return (m1+m2)*v_x + ell* m2 * v_phi*np.cos(phi)/2
def calc_conserved_quantity(q_d,v_d,params):
    I_mech = []
    for q_tmp, v_tmp in zip(q_d,v_d):
        I_mech.append(I_eval(q_tmp,v_tmp,params))
    return I_mech

conserved_I_evo = calc_conserved_quantity(q_d_vals,v_d_sol,OCP_parameters_np_version)
plt.plot(OCP_parameters_np_version["times"],conserved_I_evo)

# Standard solver

In [ ]:
def running_cost_function(u,params):
    return params["u_prefactor"]*u.dot(u)/2.

def terminal_cost(q,v,params):
    Aq = params["Aq"]
    Adq = params["Adq"]
    qT = params["qT"]
    dqT = params["dqT"]
    Delta_q = q-qT
    Delta_dq = v - dqT
    return Aq*Delta_q.dot(Delta_q) + Adq * Delta_dq.dot(Delta_dq)

def calculate_discrete_mech_dyn(U_a_d_part, U_b_d_part,params):
    q_d_vals = calculate_full_mech_trajectory(U_a_d_part,U_b_d_part)
    v_d_sol = calc_velocities_from_discrete_sol(q_d_vals,U_a_d_part,U_b_d_part,params)
    return q_d_vals,v_d_sol

def OCP_standard_def(u_d_stacked,params):
    '''supply u_d_stacked=U_a_d,U_b_d to calculate the objective function
    
    note: params not really indep of outside definition, for speed reasons already used to get compiled versions of mech dynamic
    '''
    cost = 0
    U_a_d_part, U_b_d_part = u_d_stacked.reshape([-1,1]),u_d_stacked.reshape([-1,1])
    step_size = params["h"]
    alpha = params["alpha"]
    for (ua,ub) in zip(U_a_d_part,U_b_d_part):
        cost += step_size* (alpha *running_cost_function(ua,params)+ (1-alpha)*running_cost_function(ub,params))

    q_d_vals, v_d_sol =calculate_discrete_mech_dyn(U_a_d_part,U_b_d_part,params)
    if len(q_d_vals) != len(v_d_sol):
        print("inconsistent velocity and position??")
    cost += terminal_cost(q_d_vals[-1],v_d_sol[-1],params)
    return cost



In [ ]:
stackedUs = U_a_d_values.flatten()
# OCP_standard_def(stackedUs,OCP_parameters_np_version)
ocp_standard_lambda = lambda x : OCP_standard_def(x,OCP_parameters_np_version)
standard_ocp_sol = opt.minimize(ocp_standard_lambda,stackedUs)

standard_ocp_sol = opt.minimize(ocp_standard_lambda,standard_ocp_sol.x)
Ua_standard_sol =  (standard_ocp_sol.x).reshape([-1,1])
Ub_standard_sol= Ua_standard_sol

# for faster running, do not use ocp solver, but only the new approach with an initial guess
# Ua_standard_sol,Ub_standard_sol =  U_a_d_values,U_a_d_values


q_standard_sol, v_standard_sol = calculate_discrete_mech_dyn(Ua_standard_sol,Ub_standard_sol,OCP_parameters_np_version)



In [ ]:
standard_ocp_sol = opt.minimize(ocp_standard_lambda,standard_ocp_sol.x,method='Nelder-Mead',options={"maxiter":1e6})


In [ ]:
standard_ocp_sol

In [ ]:
plt.plot(OCP_parameters_np_version["times"],np.array(q_standard_sol).transpose()[0].flatten(),label='q')
plt.plot(OCP_parameters_np_version["times"],np.array(v_standard_sol).transpose()[0].flatten(),label='v')
plt.hlines(y=0,xmin=0,xmax=OCP_parameters_np_version["times"][-1],linestyles='--')
plt.legend()
plt.show()
plt.plot(OCP_parameters_np_version["times"],np.array(q_standard_sol).transpose()[1].flatten(),label=r'$\Theta$')
plt.plot(OCP_parameters_np_version["times"],np.array(v_standard_sol).transpose()[1].flatten(),label=r'$v_\Theta$')

plt.show()
plt.plot(OCP_parameters_np_version["times"][:-1],Ua_standard_sol.flatten(),label='Ua')
plt.plot(OCP_parameters_np_version["times"][:-1],Ua_standard_sol.flatten(),'*',label='Ub')
plt.hlines(y=0,xmin=0,xmax=OCP_parameters_np_version["times"][-1])
plt.legend()

# New Lagrangian approach

In [ ]:
y = sympy.symbols("q_x,q_phi,lambda_x,lambda_phi")
v_y = sympy.symbols("v_x,v_phi,v_lambda_x,v_lambda_phi")
y_vec = sympy.Matrix(y)
v_y_vec = sympy.Matrix(v_y)
u = sympy.symbols("u_x,")
u_vec = sympy.Matrix(u)
p_y = sympy.Matrix(sympy.symbols("p_x,p_phi,p_lambda_x,p_lambda_phi"))

In [ ]:

def control_Lagrangian(y,v_y,u,params):
    q_vec,lam_vec = y[:params["dim_q"]], y[params["dim_q"]:]
    v_q_vec, v_lam_vec = v_y[:params["dim_q"]], v_y[params["dim_q"]:]
    mech_Lagrangian_func = mech_Lagrangian(q_vec,v_q_vec,params)
    f_L_func = f_L(q_vec,v_q_vec,u,params)
    result = sympy.Matrix(sympy.derive_by_array(mech_Lagrangian_func,v_q_vec)).dot(v_lam_vec)
    result += sympy.Matrix(sympy.derive_by_array(mech_Lagrangian_func,q_vec)).dot(lam_vec)
    result += f_L_func.dot(lam_vec)
    result -= running_cost_function(u,params)
    return result
#use func always for the evaluated sympy symbol 
# (probably classes would be better/more natural here)

control_Lagrangian_function = control_Lagrangian(y_vec,v_y_vec,u_vec,OCP_parameters)
p_y_vec = sympy.Matrix(sympy.derive_by_array(control_Lagrangian_function,v_y_vec))
p_control_q_vec, p_control_lambda_vec = p_y_vec[:OCP_parameters["dim_q"]], p_y_vec[OCP_parameters["dim_q"]:]
v_y_as_p_y = sympy.solve([p_y_vec-p_y],v_y)
control_inverse_Legendre_v = sympy.Matrix([v_y_as_p_y[arg] for arg in v_y])


In [ ]:
# Calculate the control Hamilton function
control_Hamilton_function = (sympy.Matrix(v_y).dot(p_y_vec) - control_Lagrangian_function).evalf(subs={tmp1:tmp2 for tmp1,tmp2 in zip(v_y,control_inverse_Legendre_v)})
control_Hamilton_function_lambdify = sympy.lambdify(y+tuple(p_y)+u, control_Hamilton_function)

In [ ]:
# Calculate nodal control from cont Lagrangian
u_optimality = sympy.derive_by_array(control_Lagrangian_function,u_vec)
# u_optimality_lambdify = sympy.lambdify(y+v_y+u,u_optimality)

u_as_y = sympy.solve(u_optimality[0],u_vec)
u_calc_from_y = [[u_as_y[tmp]] for tmp in u_as_y ]
u_calc_from_y_lambdify = sympy.lambdify(y+ v_y, u_calc_from_y)

# Discrete control Lagrangian

In [ ]:
y_d = []
U_a_d = []
U_b_d = []
for i in range(OCP_parameters["N"] + 1):
    y_k = sympy.symbols("q_x_" + str(i) + ",q_phi_"+str(i) + ",lambda_x_"+str(i)+",lambda_phi_"+str(i)+"," )
    y_d.append(sympy.Matrix(y_k))
    U_a_k = sympy.symbols("U_a_" + str(i) + ",")
    U_a_d.append(sympy.Matrix(U_a_k))
    U_b_k = sympy.symbols("U_b_" + str(i) + ",")
    U_b_d.append(sympy.Matrix(U_b_k))
U_a_d = U_a_d[:-1]
U_b_d = U_b_d[:-1]    

y_k_1 = sympy.Matrix(sympy.symbols("q_x_k_m_1,q_phi_k_m_1,lambda_x_k_m_1,lambda_phi_k_m_1"))
y_k = sympy.Matrix(sympy.symbols("q_x_k,q_phi_k,lambda_x_k,lambda_phi_k"))
y_k1 = sympy.Matrix(sympy.symbols("q_x_k_1,q_phi_k_1,lambda_x_k_1,lambda_phi_k_1"))
Ua_k_1 = sympy.Matrix(sympy.symbols("U_a_k_m_1,"))
Ua_k = sympy.Matrix(sympy.symbols("U_a_k,"))
Ub_k_1 = sympy.Matrix(sympy.symbols("U_b_k_m_1,"))
Ub_k = sympy.Matrix(sympy.symbols("U_b_k,"))


In [ ]:
def discrete_control_Lagrangian(y_k,y_k1,U_a_k,U_b_k,params):
    step_size = params["h"]
    alpha = params["alpha"]
    gamma = params["gamma"]
    y_weighted = weighted_sum(y_k,y_k1,gamma)
    y_1_weighted = weighted_sum(y_k,y_k1,1-gamma)
    y_finite_diff = finite_diff(y_k,y_k1,step_size)
    
    evaldic_yg = {x_tmp: y_tmp for (x_tmp,y_tmp) in zip(y,y_weighted)}
    evaldic_vy = {x_tmp: y_tmp for (x_tmp,y_tmp) in zip(v_y,y_finite_diff)}
    evaldic_Ua = {x_tmp:y_tmp for (x_tmp,y_tmp) in zip(u,U_a_k)}
    evaldic_y1_g = {x_tmp: y_tmp for (x_tmp,y_tmp) in zip(y,y_1_weighted)}
    evaldic_Ub = {x_tmp:y_tmp for (x_tmp,y_tmp) in zip(u,U_b_k)}
    evaldic_gammaL = evaldic_yg | evaldic_Ua | evaldic_vy 
    evaldic_1_gammaL = evaldic_y1_g | evaldic_Ub | evaldic_vy 
    result = step_size*alpha*control_Lagrangian_function.evalf(subs=evaldic_gammaL)
    result += step_size*(1-alpha)* control_Lagrangian_function.evalf(subs=evaldic_1_gammaL)
    # result = step_size * alpha * control_Lagrangian(y_weighted,y_finite_diff,U_a_k,params)
    # result += step_size * (1-alpha) * control_Lagrangian(y_1_weighted,y_finite_diff,U_b_k,params)
    return result


#Generate the discrete Lagrangians needed for the EL
evaldic_y_k_to_k_1 = {x_tmp: y_tmp for (x_tmp,y_tmp) in zip(y_k,y_k_1)}
evaldic_y_k1_to_k = {x_tmp: y_tmp for (x_tmp,y_tmp) in zip(y_k1,y_k)}
evaldic_Ua_k_to_k_1 = {x_tmp:y_tmp for (x_tmp,y_tmp) in zip(Ua_k,Ua_k_1)}
evaldic_Ub_k_to_k_1 = {x_tmp:y_tmp for (x_tmp,y_tmp) in zip(Ub_k,Ub_k_1)}
evaldic_k_to_k_1 = evaldic_y_k_to_k_1 |evaldic_y_k1_to_k |evaldic_Ua_k_to_k_1|evaldic_Ub_k_to_k_1


discrete_control_Lagrangian_func_k_k1 = discrete_control_Lagrangian(y_k,y_k1,Ua_k,Ub_k,OCP_parameters)
discrete_control_Lagrangian_func_k_1_k=discrete_control_Lagrangian_func_k_k1.evalf(subs=evaldic_k_to_k_1)    

In [ ]:
#Discrete momenta and control DEL for k -1, k, k+1 evaluation
p_control_y_km = - sympy.Matrix(sympy.derive_by_array(discrete_control_Lagrangian_func_k_k1,y_k))
p_control_y_kp = sympy.Matrix(sympy.derive_by_array(discrete_control_Lagrangian_func_k_1_k,y_k))

control_DEL_k = -p_control_y_km+  p_control_y_kp
optimality_U_a_k = sympy.derive_by_array(discrete_control_Lagrangian_func_k_k1,Ua_k)
optimality_U_b_k = sympy.derive_by_array(discrete_control_Lagrangian_func_k_k1,Ub_k)


#control velocity as function of discrete control variables
v_y_k_as_discrete_ym =control_inverse_Legendre_v.evalf(subs={ tmp_1: tmp_2 for (tmp_1,tmp_2) in zip(p_y,p_control_y_km)}| {tmp_1: tmp_2 for (tmp_1,tmp_2) in zip(y,y_k)})
v_y_k_as_discrete_yp =control_inverse_Legendre_v.evalf(subs={ tmp_1: tmp_2 for (tmp_1,tmp_2) in zip(p_y,p_control_y_kp)}| {tmp_1: tmp_2 for (tmp_1,tmp_2) in zip(y,y_k)})

p_control_y_km_lambdified = sympy.lambdify(list(y_k)+list(y_k1)+list(Ua_k) + list(Ub_k),p_control_y_km)
p_control_y_kp_lambdified = sympy.lambdify(list(y_k_1)+list(y_k)+list(Ua_k_1) + list(Ub_k_1),p_control_y_kp)



# def p_y_d_from_solution(y_d,U_a_d,U_b_d,params):
#     p_d = []
#     for tmp1,tmp2,tmp3,tmp4 in zip(y_d[:-1],y_d[1:],U_a_d,U_b_d):
#         p_d.append(p_control_y_km_lambdified(*tmp1,*tmp2,*tmp3,*tmp4))
#     p_d.append(p_control_y_kp_lambdified(*y_d[-2],*y_d[-1],*U_a_d[-1],*U_b_d[-1])) 
#     return p_d 
  
def p_y_d_from_solution(y_d,U_a_d,U_b_d,params):
    p_d = []
    p_d.append(p_control_y_km_lambdified(*y_d[0],*y_d[1],*U_a_d[0],*U_b_d[0])) 

    for tmp1,tmp2,tmp3,tmp4 in zip(y_d[:-1],y_d[1:],U_a_d,U_b_d):
        p_d.append(p_control_y_kp_lambdified(*tmp1,*tmp2,*tmp3,*tmp4))
    return p_d   


# Transform of initial q_1 from velocities

Need to be calculated numerically

In [ ]:
#calculate y_{k+1} from velocity (for initial value calculation)
#needs to be numeric again, as  not possible to explicitly calculate id

# y_k1_from_v_y_k_solver = sympy.solve(v_y_k_as_discrete_ym-sympy.Matrix(v_y), y_k1)
# y_k1_from_v_y_k = sympy.Matrix([y_k1_from_v_y_k_solver[arg] for arg in y_k1])

#todo
# calc_boundary_modifications(OCP_parameters)

## FInally calculate the full ocp necessary optimality conditions from the new approach

In [ ]:
# control_DEL_lambidified = sympy.lambdify(list(y_k_1)+list(y_k)+list(y_k1)+list(Ua_k_1)+list(Ua_k)+list( Ub_k_1)+list(Ub_k),control_DEL_k)
mu = sympy.Matrix(sympy.symbols("mu_x,mu_phi"))
nu = sympy.Matrix(sympy.symbols("nu_x,nu_phi"))

def calc_boundary_modifications(params):
    
    q_0,lambda_0 = sympy.Matrix(y_d[0][:params["dim_q"]]), sympy.Matrix(y_d[0][params["dim_q"]:])
    nu_mat = sympy.Matrix(nu)
    dict_y0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k,y_d[0])}
    dict_y1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k1,y_d[1])}
    dict_U_a_0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ua_k,U_a_d[0])}
    dict_U_b_0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ub_k,U_b_d[0])}
    full_dict = dict_y0|dict_y1| dict_U_a_0 | dict_U_b_0
    q_0,lambda_0 = sympy.Matrix(y_d[0][:params["dim_q"]]), sympy.Matrix(y_d[0][params["dim_q"]:])
    v_y_0 = v_y_k_as_discrete_ym.evalf(subs=full_dict)       
    v_q_0 = sympy.Matrix(v_y_0[:params["dim_q"]])
    D22L0 = sympy.derive_by_array(Dv_L_mech, v_q).tomatrix().subs({tmp1:tmp2 for tmp1,tmp2 in zip(v_q,v_q_0)}).subs({tmp1:tmp2 for tmp1,tmp2 in zip(y,y_d[0])}).subs({tmp1:tmp2 for tmp1,tmp2 in zip(q,y_d[0][:2])})
    eq1 = (nu_mat - D22L0@lambda_0).tolist()

    dict_yN={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k,y_d[-1])}
    dict_yN_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k_1,y_d[-2])}
    dict_U_a_N_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ua_k_1,U_a_d[-1])}
    dict_U_b_N_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ub_k_1,U_b_d[-1])}
    full_dict = dict_yN|dict_yN_1| dict_U_a_N_1 | dict_U_b_N_1
    q_N,lambda_N = sympy.Matrix(y_d[-1][:params["dim_q"]]), sympy.Matrix(y_d[-1][params["dim_q"]:])
    v_y_N = v_y_k_as_discrete_yp.evalf(subs=full_dict)
    v_q_N = sympy.Matrix(v_y_N[:params["dim_q"]])
    terminal_cont = terminal_cost(sympy.Matrix(q),sympy.Matrix(v_q),params)
    # return terminal_cont
    D_2_Phi = sympy.Matrix(sympy.derive_by_array(terminal_cont,v_q)).subs({tmp1:tmp2 for tmp1,tmp2 in zip(v_q,v_q_N)}).subs({tmp1:tmp2 for tmp1,tmp2 in zip(y,y_d[-1])}).subs({tmp1:tmp2 for tmp1,tmp2 in zip(q,y_d[-1][:2])})
    D22LN = sympy.derive_by_array(Dv_L_mech, v_q).tomatrix().subs({tmp1:tmp2 for tmp1,tmp2 in zip(v_q,v_q_N)}).subs({tmp1:tmp2 for tmp1,tmp2 in zip(y,y_d[-1])}).subs({tmp1:tmp2 for tmp1,tmp2 in zip(q,y_d[-1][:2])})
    eq2 = (D_2_Phi + D22LN@lambda_N).tolist()
    # terminal_cost(q_N,sympy.Matrix(v_y_N[:params["dim_q"]]),params)
    return eq1, eq2

def new_objective_function(y_d,U_a_d,U_b_d,mu,nu,params):
    dict_yN={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k,y_d[-1])}
    dict_yN_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k_1,y_d[-2])}
    dict_U_a_N_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ua_k_1,U_a_d[-1])}
    dict_U_b_N_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ub_k_1,U_b_d[-1])}
    full_dict = dict_yN|dict_yN_1| dict_U_a_N_1 | dict_U_b_N_1
    q_N,lambda_N = sympy.Matrix(y_d[-1][:params["dim_q"]]), sympy.Matrix(y_d[-1][params["dim_q"]:])
    v_y_N = v_y_k_as_discrete_yp.evalf(subs=full_dict)
    v_q_N = sympy.Matrix(v_y_N[:params["dim_q"]])
    objective = terminal_cost(q_N,sympy.Matrix(v_y_N[:params["dim_q"]]),params)

    for (yk_iter,yk1_iter,Uak_iter, Ubk_iter) in zip(y_d[:-1],y_d[1:],U_a_d,U_b_d):
        objective -= discrete_control_Lagrangian(yk_iter,yk1_iter,Uak_iter,Ubk_iter,params)

    dict_y0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k,y_d[0])}
    dict_y1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k1,y_d[1])}
    dict_U_a_0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ua_k,U_a_d[0])}
    dict_U_b_0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ub_k,U_b_d[0])}
    full_dict = dict_y0|dict_y1| dict_U_a_0 | dict_U_b_0
    q_0,lambda_0 = sympy.Matrix(y_d[0][:params["dim_q"]]), sympy.Matrix(y_d[0][params["dim_q"]:])
    v_y_0 = v_y_k_as_discrete_ym.evalf(subs=full_dict)       
    v_q_0 = sympy.Matrix(v_y_0[:params["dim_q"]])
    objective += mu.dot(q_0 - params["q0"]) + nu.dot(sympy.Matrix(v_y_0[:params["dim_q"]])- params["dq0"])
    # print(v_q_0)
    #add here the partial integration terms
    D2f0 = sympy.Matrix(Dv_L_mech).evalf(subs=({tmp1:tmp2 for (tmp1,tmp2) in zip(q,q_0)}|{tmp1:tmp2 for (tmp1,tmp2) in zip(v_q,v_q_0)}))
    # print(D2f0)
    D2fN = sympy.Matrix(Dv_L_mech).evalf(subs=({tmp1:tmp2 for (tmp1,tmp2) in zip(q,q_N)}|{tmp1:tmp2 for (tmp1,tmp2) in zip(v_q,v_q_N)}))

    objective -= D2f0.dot(lambda_0)
    objective += D2fN.dot(lambda_N)

    return objective

full_explicit_new_objective_function = new_objective_function(y_d,U_a_d,U_b_d,mu,nu,OCP_parameters)

def OCP_new_approach_necessary_conditions(explicit_objective_function,y_d,U_a_d,U_b_d):
    '''
    signature of the data list (each element is flattened such that e.g. y_d = (y_0,y_1,...)=(q_x_0,q_y_0, lambda_x_0, lambda_y_0,q_x_1,...)):
    y_d, U_a_d, U_b_d, mu,nu
    '''
    eqs =[]
    for tmp in y_d:
        # eqs.append(sympy.derive_by_array(explicit_objective_function,tmp).tomatrix().values())
        eqs.append(sympy.derive_by_array(full_explicit_new_objective_function,tmp).tolist())
    
    lambda_eqs = calc_boundary_modifications(OCP_parameters)
    eqs[0][2:] = lambda_eqs[0]
    eqs[-1][2:] = lambda_eqs[1]    

    for tmp in U_a_d:
        # eqs.append(sympy.derive_by_array(explicit_objective_function,tmp).tomatrix().values())
        eqs.append(sympy.derive_by_array(full_explicit_new_objective_function,tmp).tolist())


    for tmp in U_b_d:
        # eqs.append(sympy.derive_by_array(explicit_objective_function,tmp).tomatrix().values())
        eqs.append(sympy.derive_by_array(full_explicit_new_objective_function,tmp).tolist())

        
    eqs.append(sympy.derive_by_array(full_explicit_new_objective_function,mu).tolist())
    eqs.append(sympy.derive_by_array(full_explicit_new_objective_function,nu).tolist())

    # eqs.append(sympy.derive_by_array(explicit_objective_function,mu).tomatrix().values())
    # eqs.append(sympy.derive_by_array(explicit_objective_function,nu).tomatrix().values())
    
    flattened_list = []

    for tmp in eqs: 
        for tmp2 in tmp:
            flattened_list.append(tmp2[0])      
    return flattened_list

necessary_optimality_conditions_func = OCP_new_approach_necessary_conditions(full_explicit_new_objective_function,y_d,U_a_d,U_b_d)

elements = [list(tmp) for tmp in y_d]
summed_args = []
for tmp in elements:
    summed_args += tmp

elements_U_a_d = [list(tmp) for tmp in U_a_d]
for tmp in elements_U_a_d:
    summed_args+=tmp

elements_U_b_d = [list(tmp) for tmp in U_b_d]
for tmp in elements_U_b_d:
    summed_args+=tmp
summed_args += list(mu)
summed_args += list(nu)

necessary_optimality_conditions_func_lambdify = sympy.lambdify(summed_args,necessary_optimality_conditions_func)

In [ ]:
len(initial_guess_data['y_d_sol'].transpose())

In [ ]:
y_d_vals = []
U_a_d_vals = []
U_b_d_vals = []

for i in range(len(y_d)):
    if i < len(y_d)-1:
        U_a_d_vals.append(Ua_standard_sol[i])
        U_b_d_vals.append(Ub_standard_sol[i])
        y_d_vals.append(np.array([q_standard_sol[i],q_standard_sol[i]]).flatten())
    else:
        y_d_vals.append(np.array([q_standard_sol[i],q_standard_sol[i]]).flatten())
        

y_d_vals=np.array(y_d_vals)
U_a_d_vals = np.array(U_a_d_vals)
U_b_d_vals = np.array(U_b_d_vals)


    # U_a_d_vals.append(np.array([0.001]))
    # U_b_d_vals.append(np.array([0.001]))
# U_a_d_vals = U_a_d_vals[:-1]
# U_b_d_vals = U_b_d_vals[:-1]    

mu_val = np.array([5.,-45.])
nu_val = np.array([8.5,2.4])


def eval_necc_opt_condition(stacked_vec):
    crap_eval = necessary_optimality_conditions_func_lambdify(*stacked_vec)
    result_list = []
    for tmp in crap_eval:
        result_list += list(tmp.flatten())
    return  np.array(result_list)
    


In [ ]:

lam_x_func = scipy.interpolate.CubicSpline(initial_guess_data['parameters']["times"], initial_guess_data["y_d_sol"].transpose()[2].flatten()) 
lam_phi_func = scipy.interpolate.CubicSpline(initial_guess_data['parameters']["times"], initial_guess_data["y_d_sol"].transpose()[3].flatten()) 

lam_x_guess = lam_x_func(OCP_parameters_np_version["times"])
lam_phi_guess = lam_phi_func(OCP_parameters_np_version["times"])

y_d_vals = y_d_vals.transpose()
y_d_vals[2] = lam_x_guess
y_d_vals[3] = lam_phi_guess
y_d_vals = y_d_vals.transpose()
stacked_vec = np.hstack([np.array(y_d_vals).flatten(), np.array(U_a_d_vals).flatten(),np.array(U_b_d_vals).flatten(), mu_val.flatten(),nu_val.flatten()] ).flatten()


In [ ]:
def unstack_vector(stacked_vec,params):
    N_val = params["N"]
    dim_q = params["dim_q"]
    dim_u = params["dim_u"]

    y_d_vals = stacked_vec[:2*(N_val+1)*dim_q].reshape(-1,2*dim_q)
    U_a_d_vals = stacked_vec[2*(N_val+1)*dim_q: 2*(N_val+1)*dim_q + N_val*dim_u].reshape(-1,1)
    U_b_d_vals = stacked_vec[ 2*(N_val+1)*dim_q + N_val*dim_u : 2* (N_val+1)*dim_q + 2*N_val*dim_u].reshape(-1,1)
    mu_vals = stacked_vec[2* (N_val+1)*dim_q + 2*N_val*dim_u: 2* (N_val+1)*dim_q + 2*N_val*dim_u + dim_q].reshape(-1,dim_q)
    nu_vals = stacked_vec[2* (N_val+1)*dim_q + 2*N_val*dim_u  + dim_q: 2* (N_val+1)*dim_q + 2*N_val*dim_u + 2*dim_q].reshape(-1,dim_q)

    return y_d_vals,U_a_d_vals,U_b_d_vals,mu_vals,nu_vals

# testing = np.array(y_d_vals[:5])
# print(testing)
# print(testing.flatten())
# print(testing.flatten().reshape((-1,2)))

# unstack_vector(stacked_vec,OCP_parameters_np_version)[0]


## Solve here the ocp via the new method by finding solution of the necessary optimality conditions

In [ ]:
new_ocp_eval = lambda x: np.array(necessary_optimality_conditions_func_lambdify(*x))
new_ocp_result = opt.root(new_ocp_eval,stacked_vec,method='lm')
# new_ocp_result = opt.root(new_ocp_eval,stacked_vec,method='lm',tol=1e-10)


In [ ]:
# save = new_ocp_result
new_ocp_result = opt.root(new_ocp_eval,new_ocp_result.x,method='lm',tol=1e-15)


In [ ]:
new_ocp_result

## now need to extract the solution curves
we also need to calculate $$\lambda_N,\lambda_0$$ from the solution 
to correctly calculate the momenta etc

In [ ]:
def get_boundary_lambda_from_solution(params):
    dict_yN={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k,y_d[-1])}
    dict_yN_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k_1,y_d[-2])}
    dict_U_a_N_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ua_k_1,U_a_d[-1])}
    dict_U_b_N_1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ub_k_1,U_b_d[-1])}
    full_dict = dict_yN|dict_yN_1| dict_U_a_N_1 | dict_U_b_N_1 |{tmp1:tmp2 for tmp1,tmp2 in zip(q,y_d[-1][:OCP_parameters["dim_q"]])}
    q_N,lambda_N = sympy.Matrix(y_d[-1][:params["dim_q"]]), sympy.Matrix(y_d[-1][params["dim_q"]:])
    v_y_N = v_y_k_as_discrete_yp.evalf(subs=full_dict)
    v_q_N = sympy.Matrix(v_y_N[:params["dim_q"]]) 
    terminal_cost_deriv = sympy.Matrix(sympy.derive_by_array(terminal_cost(q,v_q,params),v_q)).evalf(subs={tmp1:tmp2 for (tmp1,tmp2) in zip(v_q,v_q_N)}|{tmp1:tmp2 for tmp1,tmp2 in zip(y,y_d[-1])})
    D_22_mech_L = sympy.Matrix(sympy.derive_by_array(sympy.derive_by_array(mech_Lagrangian_func,v_q),v_q)).evalf(subs={tmp1:tmp2 for (tmp1,tmp2) in zip(v_q,v_q_N)})
    eq_to_solve = terminal_cost_deriv.transpose() + lambda_N.transpose()@ D_22_mech_L
    end_solve = sympy.solve(eq_to_solve,lambda_N)
    lambda_N_vec_result = sympy.Matrix([end_solve[tmp] for tmp in lambda_N]).evalf(subs=full_dict)

    # #here the 0 side...
    dict_y0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k,y_d[0])}
    dict_y1={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(y_k_1,y_d[1])}
    dict_U_a_0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ua_k,U_a_d[0])}
    dict_U_b_0={tmp_1:tmp_2 for (tmp_1,tmp_2) in zip(Ub_k,U_b_d[0])}
    full_dict = dict_y0|dict_y1| dict_U_a_0 | dict_U_b_0 |{tmp1:tmp2 for tmp1,tmp2 in zip(q,y_d[0][:OCP_parameters["dim_q"]])}
    q_0,lambda_0 = sympy.Matrix(y_d[-1][:params["dim_q"]]), sympy.Matrix(y_d[-1][params["dim_q"]:])
    v_y_0 = v_y_k_as_discrete_ym.evalf(subs=full_dict)
    v_q_0 = sympy.Matrix(v_y_0[:params["dim_q"]])

    D_22_mech_L = sympy.Matrix(sympy.derive_by_array(sympy.derive_by_array(mech_Lagrangian_func,v_q),v_q)).evalf(subs={tmp1:tmp2 for (tmp1,tmp2) in zip(v_q,v_q_0)})
    eq_to_solve = nu.transpose() - lambda_0.transpose()@ D_22_mech_L
    start_solve = sympy.solve(eq_to_solve,lambda_0)
    lambda_0_vec_result = sympy.Matrix([start_solve[tmp] for tmp in lambda_0]).evalf(subs=full_dict)
    
    return lambda_0_vec_result, lambda_N_vec_result
    # return lambda_N_vec_result





    
boundary_lambda_func = get_boundary_lambda_from_solution(OCP_parameters_np_version)   

boundary_lambda_func_lambdify = sympy.lambdify(summed_args,boundary_lambda_func)

v_y_calc_from_q_p = sympy.lambdify(list(y)+p_y.tolist(),control_inverse_Legendre_v)


In [ ]:
#calc here all relevant soljution elements and also the initial and final lambda in its correct form
y_d_sol, u_a_d_sol, u_b_d_sol,mu_sol,nu_sol = unstack_vector(new_ocp_result.x,OCP_parameters_np_version)

#no longer needed to calculate the boundary lambdas as we have it now naturally by modifying the eqs
# boundary_lambda_calc = boundary_lambda_func_lambdify(*new_ocp_result.x)
# y_d_sol[0] = np.concatenate([y_d_sol[0][:OCP_parameters_np_version["dim_q"]],boundary_lambda_calc[0].flatten()])
# y_d_sol[-1] = np.concatenate([y_d_sol[-1][:OCP_parameters_np_version["dim_q"]],boundary_lambda_calc[1].flatten()])

p_y_d_sol = np.array(p_y_d_from_solution(y_d_sol,u_a_d_sol,u_b_d_sol,OCP_parameters_np_version))

v_d_sol = []
u_node_d_sol = []
H_control_sol = []
for tmp1,tmp2  in zip(y_d_sol,p_y_d_sol):
    v_d_sol.append(v_y_calc_from_q_p(*tmp1,*tmp2))
    u_node_d_sol.append(u_calc_from_y_lambdify(*tmp1,*v_d_sol[-1])[0])
    H_control_sol.append(control_Hamilton_function_lambdify(*tmp1,*tmp2,*u_node_d_sol[-1]))

v_d_sol = np.array(v_d_sol)
u_node_d_sol = np.array(u_node_d_sol)
H_control_sol = np.array(H_control_sol)


In [ ]:
q_standard_sol = np.array(q_standard_sol)

In [ ]:
len(q_standard_sol)

In [ ]:
plt.plot(times_vec[:-1],u_a_d_sol.flatten()-Ua_standard_sol.flatten(), label = "U_a_evo")
# plt.plot(times_vec[:-1],Ua_standard_sol.flatten(),'--', label = "standard_sol_a")

# PLot the graphs here for visualization

In [ ]:
times_vec = OCP_parameters_np_version["times"]
plt.plot(times_vec,y_d_sol.transpose()[0].flatten(),label="x_evo")
plt.plot(times_vec,q_standard_sol.transpose()[0],'--',label="standard_x_evo")
plt.legend()
plt.xlabel('time')
# plt.savefig('figs/x_evo.pdf')

plt.show()
plt.plot(times_vec,y_d_sol.transpose()[1].flatten(),label="phi_evo")
plt.plot(times_vec,q_standard_sol.transpose()[1],'--',label="standard_phi_evo")
plt.xlabel('time')
plt.legend()
# plt.savefig('figs/phi_evo.pdf')

plt.show()
plt.plot(times_vec,y_d_sol.transpose()[2].flatten(),label="lambda_x_evo")
plt.plot(times_vec,y_d_sol.transpose()[3].flatten(),label="lambda_phi_evo")
plt.xlabel('time')
plt.legend()
plt.savefig('figs/control_evo.pdf')
plt.show()
plt.plot(times_vec,p_y_d_sol.transpose()[0][0],'x',label="p_control_x_evo")
plt.xlabel('time')
plt.legend()
# plt.savefig('figs/p_control_x_evo.pdf')

plt.show()
plt.plot(times_vec,p_y_d_sol.transpose()[0][1],label="p_control_phi_evo")
plt.xlabel('time')
plt.legend()
# plt.savefig('figs/p_control_phi_evo.pdf')

plt.show()
plt.plot(times_vec,p_y_d_sol.transpose()[0][2],'x',label="p_control_lambda_x_evo")
plt.plot(times_vec,p_y_d_sol.transpose()[0][3],label="p_control_lambda_phi_evo")
plt.xlabel('time')
plt.legend()
# plt.plot(times_vec,p_y_d_sol[1].flatten(),'x',label="p_control_lambda_evo")
# plt.plot(times_vec,y_d_sol.transpose()[1].flatten(), label="lambda_evo")
# plt.legend()
# plt.savefig('figs/p_lambda_evo.pdf')

plt.show()
plt.plot(times_vec[:-1],u_a_d_sol.flatten(), label = "U_a_evo")
plt.plot(times_vec[:-1],Ua_standard_sol.flatten(),'--', label = "standard_sol_a")
plt.xlabel('time')
plt.plot(times_vec,np.array(u_node_d_sol).flatten(),'--',label='nodal controls')
plt.legend()
plt.show()
plt.plot(times_vec[:-1],u_a_d_sol.flatten()-Ua_standard_sol.flatten(), label = "delta U")

plt.show()
plt.plot(times_vec[:-1],u_b_d_sol.flatten(), label = "U_b_evo")

plt.plot(times_vec[:-1],Ub_standard_sol.flatten(),'--', label = "standard_sol_b")

plt.plot(times_vec,np.array(u_node_d_sol).flatten(),'--',label='nodal controls')
plt.xlabel('time')
plt.legend()
# plt.savefig('figs/control_evo.pdf')
plt.show()
plt.plot(times_vec,H_control_sol,label="control H")
plt.legend()


In [ ]:
plt.plot(times_vec,v_d_sol.transpose()[0][0],label='v_x')
plt.plot(times_vec,v_d_sol.transpose()[0][1],label='v_phi')
plt.legend()
plt.show()
plt.plot(times_vec,v_d_sol.transpose()[0][2],label='v_lambda_x')
plt.plot(times_vec,v_d_sol.transpose()[0][3],label='v_lambda_phi')
plt.legend()
plt.show()


In [ ]:
# plt.plot(times_vec[:-1],[x - p_y_d_sol.transpose()[0][0][0] for x in p_y_d_sol.transpose()[0][0]][:-1],'x',label="p_control_x - p_control_x(0)")
plt.plot(times_vec,[x - p_y_d_sol.transpose()[0][0][0] for x in p_y_d_sol.transpose()[0][0]],'x',label="p_control_x - p_control_x(0)")
plt.xlabel('time')
plt.legend()
# plt.savefig('figs/maybe_conserved_quantity_evo_smaller_terminal.pdf')

# plt.ylim([-0.7933,-0.794])

In [ ]:
plt.plot(y_d_sol.transpose()[0].flatten(),y_d_sol.transpose()[1].flatten())
plt.xlabel("x")
plt.ylabel('phi')
plt.show()
plt.plot(y_d_sol.transpose()[2].flatten(),y_d_sol.transpose()[3].flatten())
plt.xlabel("lam_x")
plt.ylabel('lam_phi')
plt.title("phase space diagram")

In [ ]:
#Here post processing for the consergved control quantity
# eval_p = sympy.lambdify(y+v_y,p_control_q_vec)

# def calculate_conserved_control_quantity_from_solution(y_d,)


In [ ]:
len(y_d_sol)

# Note, no need to calculate here explicitly the conserrved quantity, as it is just $p_x \leadsto$ if it is constant, then we have the onserved quantity

# Here the results will be saved in files to then have another jupyter notebook for the analysis

In [ ]:
import pickle
from pathlib import Path 


storage_dict = dict()
storage_dict["parameters"] = OCP_parameters_np_version
storage_dict["y_d_sol"] = y_d_sol
storage_dict["p_y_d_sol"] = p_y_d_sol
storage_dict["v_y_d_sol"] = v_d_sol 
storage_dict["mu_sol"] = mu_sol
storage_dict["nu_sol"] = nu_sol
storage_dict["v_standard_sol"]=v_standard_sol
storage_dict["q_standard_sol"] = q_standard_sol
storage_dict["ocp_result"] = new_ocp_result
storage_dict["u_node_d_sol"] = u_node_d_sol
storage_dict["U_a_d_sol"] = u_a_d_sol
storage_dict["U_b_d_sol"] = u_b_d_sol
storage_dict["H_control_sol"] = H_control_sol
storage_dict["standard_Ua"] = Ua_standard_sol
storage_dict["standard_Ub"] = Ub_standard_sol
storage_dict["conserved_I"] = p_y_d_sol.transpose()[0,0]
dirpath = "data/new_u_dep_data_"+"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])
Path(dirpath).mkdir(parents=True, exist_ok=True)
u_dep_file_name = dirpath+"/new_u_dep_data_" +"a=" + str(OCP_parameters_np_version["alpha"]) +"g=" + str(OCP_parameters_np_version["gamma"])   +"N=" + str(OCP_parameters_np_version["N"]) + ".pkl"
with open(u_dep_file_name, 'wb') as ffile:
    pickle.dump(storage_dict, ffile)
    ffile.close()


In [ ]:
# import os
# import sys
# sys.path.append(os.path.relpath('code' ))


# base_folder_dir = 'data'
# folder_names = os.listdir('data')
# dict_of_folders_with_files = dict()
# for x in folder_names:
#     if x != '.DS_Store':
#         dict_of_folders_with_files[x] = create_dict_for_folder(base_folder_dir+'/' +x)


# #Structure of data dict:
#     # data_dict["folder_name"]=[ {"file_name":data}, [alpha,beta,gamma,h] ]
# data_dict = dict()
# for x in dict_of_folders_with_files.keys():
#     for y in dict_of_folders_with_files[x].keys():
#         filename = base_folder_dir+"/"+x + "/"+y
#         with open(filename, 'rb') as files:
#             data_dict[y] = [pickle.load(files) ,dict_of_folders_with_files[x][y]]